# Make Dataset

In [6]:
from pathlib import Path

import pandas as pd
import seaborn as sns

from src.utils import read_json
from src.dataset import concat_dataset, shooting_score

DATA_PATH = Path("../data/")

In [7]:
players = read_json("../config/players.json")

In [8]:
skaters = concat_dataset(DATA_PATH)

Data year: 2016
Data year: 2017
Data year: 2018
Data year: 2019
Data year: 2020


In [9]:
skaters.columns.values

array(['playerId', 'season', 'name', 'team', 'position', 'situation',
       'games_played', 'icetime', 'shifts', 'gameScore',
       'onIce_xGoalsPercentage', 'offIce_xGoalsPercentage',
       'onIce_corsiPercentage', 'offIce_corsiPercentage',
       'onIce_fenwickPercentage', 'offIce_fenwickPercentage',
       'iceTimeRank', 'I_F_xOnGoal', 'I_F_xGoals', 'I_F_xRebounds',
       'I_F_xFreeze', 'I_F_xPlayStopped', 'I_F_xPlayContinuedInZone',
       'I_F_xPlayContinuedOutsideZone', 'I_F_flurryAdjustedxGoals',
       'I_F_scoreVenueAdjustedxGoals',
       'I_F_flurryScoreVenueAdjustedxGoals', 'I_F_primaryAssists',
       'I_F_secondaryAssists', 'I_F_shotsOnGoal', 'I_F_missedShots',
       'I_F_blockedShotAttempts', 'I_F_shotAttempts', 'I_F_points',
       'I_F_goals', 'I_F_rebounds', 'I_F_reboundGoals', 'I_F_freeze',
       'I_F_playStopped', 'I_F_playContinuedInZone',
       'I_F_playContinuedOutsideZone', 'I_F_savedShotsOnGoal',
       'I_F_savedUnblockedShotAttempts', 'penalties',
    

In [5]:
shooting

,season,name,shooting_score
307,2019,Artturi Lehkonen,61
337,2019,Ben Chiarot,56
480,2019,Brendan Gallagher,90
527,2019,Brett Kulak,31
857,2019,Corey Perry,44
1431,2019,Jake Evans,53
1595,2019,Jeff Petry,65
1618,2019,Jesperi Kotkaniemi,57
1660,2019,Joel Armia,76
1665,2019,Joel Edmundson,44


In [ ]:
shooting['adjusted_shooting'].apply(lambda x: int(x * 100))

In [ ]:
# Shot algo
skaters.columns.values

In [ ]:
scores = skaters[['season', 'name']]

In [ ]:
skaters[['I_F_primaryAssists', 'I_F_secondaryAssists']]

In [ ]:
skaters['goal_pct'] = skaters['I_F_goals'] / skaters['OnIce_F_shotsOnGoal']
skaters['goal_per_game'] = skaters['I_F_goals'] / skaters['games_played']
skaters['shot_per_game'] = skaters['OnIce_F_shotsOnGoal'] / skaters['games_played']
scores['shooting'] = skaters.groupby("season")[['goal_per_game', 'shot_per_game', 'goal_pct']].rank(pct=True).mean(axis=1)
#scores['adjusted_shooting'] = 

In [ ]:
adjusted_shooting = scores.groupby(["name"])['shooting'].ewm(com=0.2).mean().to_frame().reset_index().drop("level_1", axis=1)

In [ ]:
adjusted_shooting.columns = ['name_x', 'adjusted_shooting']

In [ ]:
scores = pd.concat([scores, adjusted_shooting], axis=1)
scores.drop(["name_x", "shooting"], axis=1, inplace=True)
#pd.merge(scores, adjusted_shooting, on=['name', 'season'], how='left').query("name == 'Frans Nielsen'")

In [ ]:
scores.query("name == 'Cole Caufield'")

In [ ]:
skaters.query("name=='Cole Caufield'")[['OnIce_F_shotsOnGoal', 'I_F_goals', 'goal_pct', 'games_played']]

In [ ]:
scores.sort_values("adjusted_shooting", ascending=False).head(10)

In [ ]:
mtl2020 = scores.loc[scores['name'].apply(lambda x: x in players['2020']), :].query('season==2019')
mtl2021 = scores.loc[scores['name'].apply(lambda x: x in players['2021']), :].query('season==2020')

In [ ]:
mtl2020.sort_values("adjusted_shooting")

In [ ]:
mtl2021.sort_values("adjusted_shooting")